In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
## Read data
data_path = './data'
df_anime = pd.read_csv(os.path.join(data_path, 'anime_cleaned.csv'))
df_rating = pd.read_csv(os.path.join(data_path, 'rating_cleaned_update.csv'))

In [3]:
user_id_counts = dict(df_rating['user_id'].value_counts())

In [4]:
user_id_counts

{42635: 2068,
 57620: 1996,
 59643: 1838,
 7345: 1770,
 51693: 1720,
 45659: 1665,
 65840: 1593,
 22434: 1412,
 17033: 1371,
 53698: 1363,
 11536: 1323,
 40273: 1319,
 51270: 1277,
 28196: 1218,
 54713: 1210,
 59406: 1204,
 49662: 1131,
 1530: 1127,
 23247: 1121,
 8115: 1120,
 23023: 1111,
 61110: 1111,
 47849: 1107,
 11398: 1096,
 9032: 1088,
 10419: 1084,
 58517: 1084,
 22815: 1076,
 67348: 1071,
 65836: 1068,
 56619: 1057,
 27219: 1054,
 28521: 1042,
 25497: 1037,
 41878: 1034,
 13954: 1033,
 54539: 1024,
 23975: 1021,
 21588: 1019,
 34920: 1018,
 8217: 1017,
 58233: 1016,
 52175: 1012,
 39921: 998,
 49503: 996,
 51699: 995,
 53401: 993,
 11339: 977,
 7114: 976,
 16362: 972,
 33480: 961,
 53494: 954,
 38804: 951,
 25252: 946,
 49776: 940,
 6569: 939,
 7511: 934,
 67666: 932,
 58438: 931,
 54069: 925,
 17095: 923,
 56757: 923,
 68084: 921,
 5310: 916,
 22910: 915,
 8149: 914,
 41462: 911,
 10796: 910,
 49032: 908,
 51806: 908,
 54043: 906,
 2951: 905,
 41536: 903,
 40474: 893,
 58774

### Use the 1st user as an example

In [5]:
## Save one rating dataframe and one anime dataframe for each user
user_id = list(user_id_counts.keys())[0]

df_ratings_user = df_rating[df_rating['user_id'] == user_id]
df_animes_user = df_anime[df_anime['anime_id'].isin(df_ratings_user['anime_id'])]
df_ratings_user = df_ratings_user[df_ratings_user['anime_id'].isin(df_animes_user['anime_id'])]
df_ratings_user = df_ratings_user.sort_values(by=['anime_id'])
df_animes_user = df_animes_user.sort_values(by=['anime_id'])

In [6]:
df_ratings_user

,user_id,anime_id,rating
3005992,42635,1,8
3005993,42635,5,9
3005994,42635,6,9
3005995,42635,7,7
3005996,42635,8,6
...,...,...,...
3008055,42635,32188,8
3008056,42635,32268,8
3008057,42635,32491,7
3008058,42635,32566,7


In [7]:
df_animes_user

,anime_id,name,episodes,rating,members,genre_Drama,genre_Romance,genre_School,genre_Supernatural,genre_Action,...,genre_Kids,genre_ShoujoAi,genre_Yaoi,genre_Yuri,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
22,1,Cowboy Bebop,0.013759,0.858343,0.480136,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
148,5,Cowboy Bebop: Tengoku no Tobira,0.000000,0.807923,0.135737,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
210,6,Trigun,0.013759,0.798319,0.279175,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2001,7,Witch Hunter Robin,0.013759,0.683073,0.064003,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2967,8,Beet the Vandel Buster,0.028068,0.647059,0.009701,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,32188,Steins;Gate: Kyoukaimenjou no Missing Link - D...,0.000000,0.800720,0.037612,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,1,0
891,32268,Koyomimonogatari,0.006054,0.732293,0.044994,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,0
928,32491,Kanojo to Kanojo no Neko: Everything Flows,0.001651,0.729892,0.038222,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
1272,32566,Noblesse: Pamyeol-ui Sijak,0.000000,0.710684,0.012236,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0


In [8]:
## 80-20 train test split
X = df_animes_user.iloc[:, 2:]
Y = df_ratings_user.iloc[:, -1]

n_samples = X.shape[0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    train_size=int(n_samples * 0.8), 
                                                    shuffle=True, 
                                                    random_state=0)

In [9]:
## Apply PCA on anime data to reduce to n_pca dimensions
n_pca = min(20, len(X_train))
print(n_pca)

pca_X_train = PCA(n_components=n_pca)
pca_X_train.fit(X_train)
print('PCA explained variance ratio: ', sum(pca_X_train.explained_variance_ratio_))

X_train = pca_X_train.transform(X_train)
X_test = pca_X_train.transform(X_test)

20
PCA explained variance ratio:  0.8340755844310613


In [10]:
## Access the existing ratings scored by the user. New ratings are predicted based on the existing ratings.
rating_counts = dict(Y_train.value_counts())
existing_ratings = np.array(list(rating_counts.keys()))
existing_rating_counts = np.array(list(rating_counts.values()))

print('Existing Ratings: ', existing_ratings)
print('Existing Rating Counts: ', existing_rating_counts)

## Filter the existing ratings based on their occurrences
existing_ratings = existing_ratings[existing_rating_counts >= 2]

Existing Ratings:  [ 7  6  5  8  9 10]
Existing Rating Counts:  [613 518 243 242  29   9]


In [11]:
## sample_means records the mean anime attributes for each rating
## sample_covs records the covariance matrix of anime attributes for each rating
sample_means = np.zeros((len(existing_ratings), n_pca))
sample_covs = np.zeros((len(existing_ratings), n_pca, n_pca))

for j, rating in enumerate(existing_ratings):
    X_train_sub = X_train[Y_train == rating, :]
    
    sample_mean = np.mean(X_train_sub, axis=0)
    sample_means[j, :] = sample_mean
    
    sample_cov = np.cov(X_train_sub.T)
    sample_covs[j, :] = sample_cov
    
    print(sample_mean.shape, ' ', sample_cov.shape)

(20,)   (20, 20)
(20,)   (20, 20)
(20,)   (20, 20)
(20,)   (20, 20)
(20,)   (20, 20)
(20,)   (20, 20)


In [12]:
## Assign to test data the rating with the smallest Mahalanobis distance
Y_res = np.zeros_like(Y_test)

for j, x in enumerate(X_test):
    mahalanobis_distances = np.zeros(len(existing_ratings))
    mahalanobis_valid = True
    
    for k in range(len(existing_ratings)):
        x_zero_meaned = x - sample_means[k]
        
        try:
            mahalanobis_distances[k] = np.abs(x_zero_meaned @ np.linalg.inv(sample_covs[k]) @ x_zero_meaned.T)
        except:
            print('[Warning] Problem encountered in calculation of mahalanobis distance. Using pseudo-inverse instead.')
            mahalanobis_distances[k] = np.abs(x_zero_meaned @ np.linalg.pinv(sample_covs[k]) @ x_zero_meaned.T)
            
    
    Y_res[j] = existing_ratings[np.argmin(mahalanobis_distances)]
    
    print('------------------------------')
    print('mahalanobis_distances: ', mahalanobis_distances)
    print('answered: ', Y_res[j])
    print('correct: ', Y_test.iloc[j])
    print('------------------------------')
    
mse = np.mean(np.square(Y_test - Y_res))
rmse = np.sqrt(mse)
print('[User #0] MSE: ', mse, ' RMSE: ', rmse)

------------------------------
mahalanobis_distances:  [3.40195048e+01 3.71178630e+01 3.23893387e+01 3.61402498e+01
 3.08565555e+03 4.44126044e+17]
answered:  5
correct:  5
------------------------------
------------------------------
mahalanobis_distances:  [1.43686748e+01 1.25739100e+01 1.50090294e+01 1.71980522e+01
 5.20143816e+01 1.15853148e+17]
answered:  6
correct:  6
------------------------------
------------------------------
mahalanobis_distances:  [1.78880035e+01 2.84731260e+01 6.81038861e+01 2.87267649e+01
 7.99690534e+01 4.68306830e+15]
answered:  7
correct:  7
------------------------------
------------------------------
mahalanobis_distances:  [1.13135004e+01 1.12189139e+01 1.14934703e+01 1.37077421e+01
 1.22792608e+02 4.02410038e+16]
answered:  6
correct:  7
------------------------------
------------------------------
mahalanobis_distances:  [1.16993978e+01 1.39324909e+01 1.75313717e+01 1.05516612e+01
 6.78626222e+01 2.68177691e+17]
answered:  8
correct:  8
-----------

------------------------------
mahalanobis_distances:  [5.11207064e+01 6.19888186e+01 1.17746214e+02 5.01183816e+01
 3.81534955e+02 4.62713202e+16]
answered:  8
correct:  8
------------------------------
------------------------------
mahalanobis_distances:  [1.78776445e+01 1.53609763e+01 2.07907248e+01 1.06964035e+01
 7.76026674e+01 5.56825979e+16]
answered:  8
correct:  6
------------------------------
------------------------------
mahalanobis_distances:  [3.65691848e+01 3.96258291e+01 7.63673482e+01 3.80914345e+01
 2.82597989e+03 4.89441969e+17]
answered:  7
correct:  7
------------------------------
------------------------------
mahalanobis_distances:  [1.45832232e+01 1.69625957e+01 3.07896007e+01 1.31705611e+01
 6.82749675e+01 7.73221156e+16]
answered:  8
correct:  9
------------------------------
------------------------------
mahalanobis_distances:  [1.50850285e+01 1.60122878e+01 2.59910921e+01 1.60285567e+01
 4.82018796e+01 1.35406588e+16]
answered:  7
correct:  7
-----------

 8.53511094e+01 6.60283408e+15]
answered:  8
correct:  9
------------------------------
------------------------------
mahalanobis_distances:  [1.57128862e+01 1.95385272e+01 3.21935922e+01 1.52859610e+01
 6.53282554e+01 2.71065926e+17]
answered:  8
correct:  7
------------------------------
------------------------------
mahalanobis_distances:  [2.01167197e+01 2.07321373e+01 2.98707431e+01 2.05379847e+01
 6.73023151e+01 2.69405808e+16]
answered:  7
correct:  5
------------------------------
------------------------------
mahalanobis_distances:  [1.84976672e+01 1.93767462e+01 2.22930068e+01 3.71122909e+01
 2.06042485e+04 2.74060780e+17]
answered:  7
correct:  5
------------------------------
------------------------------
mahalanobis_distances:  [2.18450210e+01 3.01059027e+01 6.79934472e+01 3.31639869e+01
 2.10473565e+04 1.33089659e+17]
answered:  7
correct:  8
------------------------------
------------------------------
mahalanobis_distances:  [1.25956861e+01 1.12282109e+01 9.40748772

### Apply the above process to all users

In [13]:
## Save intermediate data
pca_var_rate = []
mse_all = []
rmse_all = []

In [14]:
for i, user_id in enumerate(tqdm(user_id_counts.keys())):
    #if i > 100:
    #    break
    
    #if i < 50000:
    #    continue
        
    ## Save one rating dataframe and one anime dataframe for every user
    df_ratings_user = df_rating[df_rating['user_id'] == user_id]
    df_animes_user = df_anime[df_anime['anime_id'].isin(df_ratings_user['anime_id'])]
    df_ratings_user = df_ratings_user[df_ratings_user['anime_id'].isin(df_animes_user['anime_id'])]
    df_ratings_user = df_ratings_user.sort_values(by=['anime_id'])
    df_animes_user = df_animes_user.sort_values(by=['anime_id'])
    
    ## 80-20 train test split
    X = df_animes_user.iloc[:, 2:]
    Y = df_ratings_user.iloc[:, -1]
    
    n_samples = X.shape[0]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                        train_size=int(n_samples * 0.8), 
                                                        shuffle=True, 
                                                        random_state=0)
    
    ## Apply PCA on anime data to reduce to n_pca dimensions
    n_pca = min(20, len(X_train))
    
    pca_X_train = PCA(n_components=n_pca)
    pca_X_train.fit(X_train)
    
    pca_var_rate.append(sum(pca_X_train.explained_variance_ratio_))
    
    X_train = pca_X_train.transform(X_train)
    X_test = pca_X_train.transform(X_test)
    
    ## Access the existing ratings scored by the user. New ratings are predicted based on the existing ratings.
    rating_counts = dict(Y_train.value_counts())
    existing_ratings = np.array(list(rating_counts.keys()))
    existing_rating_counts = np.array(list(rating_counts.values()))
    
    ## Filter the existing ratings based on their occurrences
    existing_ratings = existing_ratings[existing_rating_counts >= 2]
    
    ## sample_means records the mean anime attributes for each rating
    ## sample_covs records the covariance matrix of anime attributes for each rating    
    sample_means = np.zeros((len(existing_ratings), n_pca))
    sample_covs = np.zeros((len(existing_ratings), n_pca, n_pca))
    
    for j, rating in enumerate(existing_ratings):
        X_train_sub = X_train[Y_train == rating, :]
        
        sample_mean = np.mean(X_train_sub, axis=0)
        sample_means[j, :] = sample_mean
        
        sample_cov = np.cov(X_train_sub.T)
        sample_covs[j, :] = sample_cov
        
    ## Assign to test data the rating with the smallest Mahalanobis distance
    Y_res = np.zeros_like(Y_test)
    
    for j, x in enumerate(X_test):
        mahalanobis_distances = np.zeros(len(existing_ratings))
        
        for k in range(len(existing_ratings)):
            x_zero_meaned = x - sample_means[k]
            
            try:
                mahalanobis_distances[k] = np.abs(x_zero_meaned @ np.linalg.inv(sample_covs[k]) @ x_zero_meaned.T)
            except:
                #print('[Warning] Problem encountered in calculation of mahalanobis distance. Using pseudo-inverse instead.')
                mahalanobis_distances[k] = np.abs(x_zero_meaned @ np.linalg.pinv(sample_covs[k]) @ x_zero_meaned.T)
        
        Y_res[j] = existing_ratings[np.argmin(mahalanobis_distances)]
        
    mse = np.mean(np.square(Y_test - Y_res))
    rmse = np.sqrt(mse)
    #print('[User #', i, '] MSE: ', mse, ' RMSE: ', rmse)
    
    mse_all.append(mse)
    rmse_all.append(rmse)

100%|██████████| 54004/54004 [12:17<00:00, 73.27it/s]


In [15]:
user_id_counts = np.array(list(user_id_counts.values()))
user_id_counts = user_id_counts.reshape(len(user_id_counts), 1)

pca_var_rate = np.array(pca_var_rate).reshape(len(pca_var_rate), 1)
mse_all = np.array(mse_all).reshape(len(mse_all), 1)
rmse_all = np.array(rmse_all).reshape(len(rmse_all), 1)

print('Average PCA-explained variance ratio over all users: ', np.mean(pca_var_rate))
print('Average MSE over all users: ', np.mean(mse_all))
print('Average RMSE over all users: ', np.mean(rmse_all))

Average PCA-explained variance ratio over all users:  0.9574022880664051
Average MSE over all users:  2.538507664388136
Average RMSE over all users:  1.4590805840710457


In [16]:
outfile = './results/user_space_pca_res.npz'
np.savez(outfile, user_id_counts, pca_var_rate, mse_all, rmse_all)